In [1]:
%pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is 

Uncomment if working on colab

In [4]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [57]:
from datasets import Dataset, ClassLabel, Features, Sequence, Value, DatasetDict


import json

with open("/content/drive/MyDrive/save_good3.json", 'r', encoding='utf-8') as file:
    # Load the JSON data from the file
    data = json.load(file)

# data['id'] = data.pop('ids')
# ner_feature = ClassLabel(names=['O', 'FURNITURE'])
data['ner_tags'] = [0 if tag == 'O' else 1 for tag in data['ner_tags']]


# features = Features({
#     'text': Sequence(feature=Value('string')),
#     'label': Sequence(feature=ner_feature)
#     # Add other features if needed
# })

# Create the Dataset with specified features
first_dataset = Dataset.from_dict({
    'text': data['tokens'],
    'label': data['ner_tags']
})

In [58]:

# raw_datasets = raw_datasets.cast_column("ner_tags", ner_feature)
train_dataset, temp_dataset = first_dataset.train_test_split(test_size=0.75).values()

# Further split the temp dataset into validation and test
validation_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.66).values()

raw_datasets = DatasetDict({
    'train': train_dataset,
    'test': validation_dataset,
    'unsupervised': test_dataset
})

print(raw_datasets)

# ner_feature = raw_datasets['train'].features["ner_tags"]
# print(ner_feature)
# # ner_feature = raw_datasets['validation'].features["tokens"]
# # print(ner_feature)
# # ner_feature = raw_datasets['test'].features["id"]
# # print(ner_feature)
# label_names = ner_feature.feature.names
# label_names



DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4624
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4716
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 9156
    })
})


In [59]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [60]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=512)

In [61]:
tokenized_tags = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/4624 [00:00<?, ? examples/s]

Map:   0%|          | 0/4716 [00:00<?, ? examples/s]

Map:   0%|          | 0/9156 [00:00<?, ? examples/s]

In [62]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [63]:
import evaluate

accuracy = evaluate.load("accuracy")

In [64]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [65]:
id2label = {0: "O", 1: "FURNITURE"}
label2id = {"O": 0, "FURNITURE": 1}

In [93]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [94]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_tags["train"],
    eval_dataset=tokenized_tags["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.302203,0.880407
2,0.293000,0.300918,0.884012
3,0.293000,0.321598,0.901187


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.302203,0.880407
2,0.293000,0.300918,0.884012
3,0.293000,0.321598,0.901187
4,0.159600,0.332574,0.906913


TrainOutput(global_step=1156, training_loss=0.20923296968004695, metrics={'train_runtime': 1156.326, 'train_samples_per_second': 15.995, 'train_steps_per_second': 1.0, 'total_flos': 2450117005541376.0, 'train_loss': 0.20923296968004695, 'epoch': 4.0})

In [3]:
text1 = "<a _ngcontent-ng-c4105231812=\"\" class=\"link\" href=\"/about/privacy-statement\">PRIVACY STATEMENT</a>"
text2 = "<a class=\"announcement-bar__link link link--text focus-inset animate-arrow\" href=\"/pages/mattresses\"> <img alt=\"\" src=\"//www.factorybuys.com.au/cdn/shop/files/thumbs-up_449b5a70-dff2-4a1c-acff-bdf45491b079.png?v=1659573667\"/> <span class=\"announcement-bar__message\"> FREE SHIPPING ON MATTRESSES* </span></a>"

In [4]:
from transformers import pipeline

classifier = pipeline("text-classification", model="checkpoint-1156")
print(classifier(text1)[0]['label'])
print(classifier(text2))

O
[{'label': 'FURNITURE', 'score': 0.9845311641693115}]


In [33]:
import requests
from bs4 import BeautifulSoup

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

tags_to_keep = ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span', 'strong', 'em', 'a']

def filter_html_by_tags(soup, tags_to_keep):
    # Create a new BeautifulSoup object to store the filtered HTML
    filtered_soup = []

    # Iterate over all the tags in the soup
    for tag in soup.find_all(True):  # True finds all tags
        if tag.name in tags_to_keep:
            # Copy the tag and its contents to the filtered_soup
            filtered_soup.append(str(tag).replace('  ', '').replace('\n', ' '))

    return filtered_soup

def copy_web_page(url):
    try:
        # Attempt to fetch the web page
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes

        # Parse the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove script and style elements
        for script in soup(['script', 'style']):
            script.extract()

        # Filter the soup to keep only the specified tags and their contents
        filtered_soup = filter_html_by_tags(soup, tags_to_keep)

        # Save the filtered HTML to a file
        # with open(f"{output_directory}/split_htmls.txt", "w", encoding="utf-8") as file:
        #     file.write(filtered_soup.prettify())
        print(f"Successfully saved {url}")

        return filtered_soup


    except requests.exceptions.RequestException as e:
        # Handle any exceptions related to the request
        print(f"Failed to retrieve {url}: {e}")

        return []

In [34]:
from bs4 import BeautifulSoup

def remove_path_tags(html):
    soup = BeautifulSoup(html, 'html.parser')

    # Find all <svg> elements
    for svg in soup.find_all('svg'):
        # Remove all <path> tags within each <svg>
        for path in svg.find_all('path'):
            path.decompose()

    # Return cleaned HTML
    return str(soup)

def remove_empty_tags(tokens):
    filtered_indexes = []

    for indx, html in enumerate(tokens):
        soup = BeautifulSoup(html, 'lxml')
        if soup.get_text(strip=True):  # Check if there is any non-whitespace text
            filtered_indexes.append(indx)

    new_tokens = [remove_path_tags(tokens[i]) for i in filtered_indexes]

    return new_tokens


In [35]:
def parse_html(html_string):    
    # Parse the HTML
    soup = BeautifulSoup(html_string, 'html.parser')

    return soup.get_text()

In [36]:
from tqdm import tqdm

def process_link(link):
  tokens = remove_empty_tags(copy_web_page(link))

  processed_list = []
  if len(tokens) == 0:
    return {'link': link, 'result': 'No furniture found'}

  for i in tqdm(range(len(tokens))):
    if classifier(tokens[i], truncation=True)[0]['label'] == 'FURNITURE':
      processed_list.append(parse_html(tokens[i]))

  elem = {'link': link, 'tokens': processed_list}
  return elem

In [40]:
import pandas as pd
import json

with open('result.json', 'r', encoding='utf-8') as f:
    elems = json.load(f)

# Read the furniture file
links = pd.read_csv('furniture_stores_pages.csv')

for index, link in enumerate(links['max(page)'][81:]):
    elem = process_link(link)
    elems.append(elem)
    print(f"Finished processing link {index + 1}")
    
    # Save the prime_dataset to a JSON file
    json_output_file = 'result.json'
    with open(json_output_file, 'w', encoding='utf-8') as f:
        json.dump(elems, f, ensure_ascii=False, indent=4)


Successfully saved https://www.warmnordic.com/global/products/news


100%|██████████| 835/835 [00:42<00:00, 19.82it/s]


Finished processing link 1
Successfully saved https://www.royaloakfurniture.co.uk/products/pop-bench


100%|██████████| 145/145 [00:07<00:00, 20.08it/s]


Finished processing link 2
Successfully saved https://mulamu.com/products/membership


...

Finished processing link 619
Failed to retrieve http://aonefurniture.in/userdata/products/no%20value: 522 Server Error:  for url: https://aonefurniture.in/userdata/products/no%20value
Finished processing link 620
Successfully saved https://furnituremama.com/products/three-door-wardrobe-mirror-2


100%|██████████| 80/80 [00:05<00:00, 14.78it/s]


Finished processing link 621
Successfully saved https://gfurniture.ca/products/


100%|██████████| 210/210 [00:07<00:00, 26.96it/s]


Finished processing link 622
Failed to retrieve https://www.mwfurnitureoutlet.com/products/dresser: 404 Client Error: Not Found for url: https://luxhomedecor.com/products/dresser
Finished processing link 623
